In [58]:
import pandas as pd
import matplotlib.pylab as plt
import matplotlib
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [59]:
OutPut = pd.DataFrame(columns = ['Year','cost_of_gas','cost_of_EV','cost_of_PHEV','cost_of_FCV'])
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV


# Dataset in 2022

In [60]:
df_gas = pd.read_excel('../Raw_Data/2022 FE Guide for DOE-release dates before 4-16-2022-no-sales -4-16-2022public.xlsx', sheet_name = '22')
df_EV = pd.read_excel('../Raw_Data/2022 FE Guide for DOE-release dates before 4-16-2022-no-sales -4-16-2022public.xlsx', sheet_name = '22 EV')
df_PHEV = pd.read_excel('../Raw_Data/2022 FE Guide for DOE-release dates before 4-16-2022-no-sales -4-16-2022public.xlsx', sheet_name = '22 PHEV')
df_FCV = pd.read_excel('../Raw_Data/2022 FE Guide for DOE-release dates before 4-16-2022-no-sales -4-16-2022public.xlsx', sheet_name = '22 FCV')

In [61]:
df_gas.head()

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,City FE (Guide) - Conventional Fuel,...,City CO2 Rounded Adjusted,Hwy CO2 Rounded Adjusted,Comb CO2 Rounded Adjusted (as shown on FE Label),CO2-PHEV Composite Rounded Adjusted Combined CO2,240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 10 miles)DISTANCE,City PHEV Composite MPGe,Hwy PHEV Composite MPGe,Comb PHEV Composite MPGe
0,2022,Honda,Acura,NSX,HNX,521,3.5,6,Auto(AM-S9),21,...,420,406,420,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,aston martin,Aston Martin Lagonda Ltd,Vantage V8,ASX,4,4.0,8,Auto(A8),18,...,494,364,435,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,Volkswagen Group of,Audi,R8 Coupe,VGA,14,5.2,10,Auto(AM-S7),13,...,670,453,572,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,Volkswagen Group of,Audi,R8 Coupe RWD,VGA,16,5.2,10,Auto(AM-S7),14,...,627,385,518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,Volkswagen Group of,Audi,R8 Spyder,VGA,15,5.2,10,Auto(AM-S7),13,...,670,453,572,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_EV.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Model Type Information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charge Depleting Information (Electricity Oper...,...,Battery Charge Time,NaN,Range,PHEV Composite MPG,NaN,NaN,Electric Driving Range,NaN,NaN,NaN
4,Model Yr,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,City FE (Guide) - Conventional Fuel,...,240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 1...,City PHEV Composite MPGe,Hwy PHEV Composite MPGe,Comb PHEV Composite MPGe,City Range (miles),Hwy Range (miles),Comb Range as shown on FE Label (miles),


In [63]:
df_PHEV.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,(Not Confidential),Unnamed: 9,...,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Model Type Information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charge Sustaining Information (Gasoline Opera...,...,Charge Depleting Driving Range,NaN,NaN,Individual Utility Factors (Rcda MDIUFs),NaN,NaN,PHEV Charge Depleting Fuel Consumption,NaN,NaN,Fuel Tank
3,Model Yr (gold fill means release date is aft...,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,City FE (Guide) - Conventional Fuel,...,City Range on Electricity (miles),Hwy Range on Electricity (miles),Comb Range on Electricity as shown on FE Label...,City Utility Factor (MDIUF),Hwy Utility Factor (MDIUF),Comb Utility Factor (MDIUF),City (gal/100 mi),Hwy (gal/100 mi),Combined (gal/100 mi) as listed on blended PHE...,Size (gal)
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_FCV.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Total,NaN,NaN,NaN,NaN,NaN,NaN
1,Model Type Information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charge Depleting Information (Electricity Oper...,...,PHEV,Battery Charge Time,NaN,Range,PHEV Composite MPG,NaN,NaN,Electric Driving Range,NaN,NaN
2,Model Yr,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,City FE (Guide) - Conventional Fuel,...,CO2-PHEV Composite Rounded Adjusted Combined CO2,240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 1...,City PHEV Composite MPGe,Hwy PHEV Composite MPGe,Comb PHEV Composite MPGe,City Range (miles),Hwy Range (miles),Comb Range as shown on FE Label (miles)
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_EV['Unnamed: 44']

0                                        NaN
1                                        NaN
2                                        NaN
3                              Ann Fuel Cost
4      Annual Fuel1 Cost - Conventional Fuel
                       ...                  
249                                      NaN
250                                      NaN
251                                      NaN
252                                      NaN
253                                      NaN
Name: Unnamed: 44, Length: 254, dtype: object

In [66]:
df_FCV['Unnamed: 44']

0                                       NaN
1                             Ann Fuel Cost
2     Annual Fuel1 Cost - Conventional Fuel
3                                       NaN
4                                       NaN
5                                       NaN
6                                      1500
7                                      1500
8                                       NaN
9                                      1400
10                                     1400
11                                      NaN
12                                     1150
13                                     1150
14                                      NaN
15                                     1300
16                                     1300
17                                      NaN
18                                     1150
19                                     1150
Name: Unnamed: 44, dtype: object

In [67]:
df_PHEV['Unnamed: 44']

0                                        NaN
1                                        NaN
2                              Ann Fuel Cost
3      Annual Fuel1 Cost - Conventional Fuel
4                                        NaN
                       ...                  
138                                      NaN
139                                      NaN
140                                      NaN
141                                      NaN
142                                      NaN
Name: Unnamed: 44, Length: 143, dtype: object

In [68]:
df_EV = df_EV.drop(df_EV.index[[3,4]])

In [69]:
df_PHEV = df_PHEV.drop(df_PHEV.index[[2,3]])

In [70]:
df_FCV = df_FCV.drop(df_FCV.index[[1,2]])

In [71]:
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 44'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()

In [72]:
cost_of_gas, cost_of_EV, cost_of_PHEV, cost_of_FCV

(1859.9056603773586, 698.125, 1318.6046511627908, 1300.0)

In [73]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV


In [74]:
OutPut = OutPut.append({'Year':2022, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.90566,698.125,1318.604651,1300.0


# Dataset in 2021

In [75]:
df_gas = pd.read_excel('../Raw_Data/2021 FE Guide-release dates before 11-23-2021-no-sales -11-22-2021 for DOE_Karmapublic.xlsx', sheet_name = '21')
df_EV = pd.read_excel('../Raw_Data/2021 FE Guide-release dates before 11-23-2021-no-sales -11-22-2021 for DOE_Karmapublic.xlsx', sheet_name = '21 EV')
df_PHEV = pd.read_excel('../Raw_Data/2021 FE Guide-release dates before 11-23-2021-no-sales -11-22-2021 for DOE_Karmapublic.xlsx', sheet_name = '21 PHEV')
df_FCV = pd.read_excel('../Raw_Data/2021 FE Guide-release dates before 11-23-2021-no-sales -11-22-2021 for DOE_Karmapublic.xlsx', sheet_name = '21 FCV')

In [76]:
df_EV['Unnamed: 44']

0                                        NaN
1                                        NaN
2                              Ann Fuel Cost
3      Annual Fuel1 Cost - Conventional Fuel
4                                        NaN
                       ...                  
156                                      850
157                                      850
158                                      NaN
159                                      NaN
160                                      NaN
Name: Unnamed: 44, Length: 161, dtype: object

In [77]:
df_PHEV['Unnamed: 44']

0                                        NaN
1                                        NaN
2                                        NaN
3                              Ann Fuel Cost
4      Annual Fuel1 Cost - Conventional Fuel
                       ...                  
165                                      NaN
166                                      NaN
167                                      NaN
168                                      NaN
169                                      NaN
Name: Unnamed: 44, Length: 170, dtype: object

In [78]:
df_FCV['Unnamed: 44']

0                                       NaN
1                                       NaN
2                             Ann Fuel Cost
3     Annual Fuel1 Cost - Conventional Fuel
4                                       NaN
5                                       NaN
6                                       NaN
7                                      1250
8                                      1250
9                                       NaN
10                                      NaN
11                                     1500
12                                     1500
13                                      NaN
14                                      NaN
15                                     1400
16                                     1400
17                                      NaN
18                                     1300
19                                     1300
20                                      NaN
21                                     1150
22                              

In [79]:
df_EV = df_EV.drop(df_EV.index[[2,3]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[3,4]])
df_FCV = df_FCV.drop(df_FCV.index[[2,3]])
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 44'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()
OutPut = OutPut.append({'Year':2021, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

In [80]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.905660,698.125000,1318.604651,1300.0
1,2021.0,2095.423729,695.098039,1492.000000,1320.0


# Dataset in 2020

In [81]:
df_gas = pd.read_excel('../Raw_Data/2020 FE Guide-adds and corrections for DOE-OK for release-no-sales- 4-7-2021Koenigseggpublic.xlsx', sheet_name = '20')
df_EV = pd.read_excel('../Raw_Data/2020 FE Guide-adds and corrections for DOE-OK for release-no-sales- 4-7-2021Koenigseggpublic.xlsx', sheet_name = '20 EV')
df_PHEV = pd.read_excel('../Raw_Data/2020 FE Guide-adds and corrections for DOE-OK for release-no-sales- 4-7-2021Koenigseggpublic.xlsx', sheet_name = '20 PHEV')
df_FCV = pd.read_excel('../Raw_Data/2020 FE Guide-adds and corrections for DOE-OK for release-no-sales- 4-7-2021Koenigseggpublic.xlsx', sheet_name = '20 FCV')

In [82]:
df_EV['Unnamed: 44']

0                                        NaN
1                              Ann Fuel Cost
2      Annual Fuel1 Cost - Conventional Fuel
3                                        NaN
4                                        NaN
                       ...                  
113                                      550
114                                      550
115                                      NaN
116                                      600
117                                      600
Name: Unnamed: 44, Length: 118, dtype: object

In [83]:
df_PHEV['Unnamed: 44'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                      NaN
8                                      NaN
9                                      NaN
Name: Unnamed: 44, dtype: object

In [84]:
df_FCV['Unnamed: 44']

0                                       NaN
1                                       NaN
2                                       NaN
3                             Ann Fuel Cost
4     Annual Fuel1 Cost - Conventional Fuel
5                                       NaN
6                                       NaN
7                                       NaN
8                                      1250
9                                      1250
10                                      NaN
11                                     1500
12                                     1500
13                                      NaN
14                                     1400
15                                     1400
16                                      NaN
17                                     1250
18                                     1250
19                                      NaN
20                                      NaN
21                                      NaN
22                              

In [85]:
df_EV = df_EV.drop(df_EV.index[[1,2]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[4,5]])
df_FCV = df_FCV.drop(df_FCV.index[[3,4]])
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 44'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()
OutPut = OutPut.append({'Year':2020, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

In [86]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.905660,698.125000,1318.604651,1300.0
1,2021.0,2095.423729,695.098039,1492.000000,1320.0
2,2020.0,2076.817793,646.052632,1445.555556,1350.0


# Dataset in 2019

In [87]:
df_gas = pd.read_excel('../Raw_Data/2019 FE Guide-for DOE-release dates before 12-19-2019-no-sales- 12_17_2019Koenigseggpublic.xlsx', sheet_name = '2019 FEguide')
df_EV = pd.read_excel('../Raw_Data/2019 FE Guide-for DOE-release dates before 12-19-2019-no-sales- 12_17_2019Koenigseggpublic.xlsx', sheet_name = 'EVs')
df_PHEV = pd.read_excel('../Raw_Data/2019 FE Guide-for DOE-release dates before 12-19-2019-no-sales- 12_17_2019Koenigseggpublic.xlsx', sheet_name = 'PHEVs')
df_FCV = pd.read_excel('../Raw_Data/2019 FE Guide-for DOE-release dates before 12-19-2019-no-sales- 12_17_2019Koenigseggpublic.xlsx', sheet_name = 'FCVs')

In [88]:
df_EV['Unnamed: 44'].head(8)

0                                      NaN
1                                      NaN
2                            Ann Fuel Cost
3    Annual Fuel1 Cost - Conventional Fuel
4                                      NaN
5                                      NaN
6                                      900
7                                      900
Name: Unnamed: 44, dtype: object

In [89]:
df_PHEV['Unnamed: 44'].head(8)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                      NaN
Name: Unnamed: 44, dtype: object

In [90]:
df_FCV['Unnamed: 44']

0                                       NaN
1                             Ann Fuel Cost
2     Annual Fuel1 Cost - Conventional Fuel
3                                       NaN
4                                       NaN
5                                      1250
6                                      1250
7                                       NaN
8                                      1500
9                                      1500
10                                      NaN
11                                     1400
12                                     1400
13                                      NaN
14                                     1250
15                                     1250
16                                      NaN
17                                      NaN
18                                      NaN
19                                      NaN
20                                      NaN
21                                      NaN
Name: Unnamed: 44, dtype: object

In [91]:
df_EV = df_EV.drop(df_EV.index[[2,3]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[4,5]])
df_FCV = df_FCV.drop(df_FCV.index[[1,2]])
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 44'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()
OutPut = OutPut.append({'Year':2019, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

In [92]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.905660,698.125000,1318.604651,1300.0
1,2021.0,2095.423729,695.098039,1492.000000,1320.0
2,2020.0,2076.817793,646.052632,1445.555556,1350.0
3,2019.0,1904.559748,628.571429,1258.333333,1350.0


# Dataset in 2018

In [93]:
df_gas = pd.read_excel('../Raw_Data/2018 FE Guide for DOE3 -all rel dates-no-sales-3-25-2019McLarenforpublic.xlsx', sheet_name = 'FEguide')
df_EV = pd.read_excel('../Raw_Data/2018 FE Guide for DOE3 -all rel dates-no-sales-3-25-2019McLarenforpublic.xlsx', sheet_name = 'EVs')
df_PHEV = pd.read_excel('../Raw_Data/2018 FE Guide for DOE3 -all rel dates-no-sales-3-25-2019McLarenforpublic.xlsx', sheet_name = 'PHEVs')
df_FCV = pd.read_excel('../Raw_Data/2018 FE Guide for DOE3 -all rel dates-no-sales-3-25-2019McLarenforpublic.xlsx', sheet_name = 'FCVs')

In [94]:
df_EV['Unnamed: 44'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                      NaN
8                                      550
9                                      550
Name: Unnamed: 44, dtype: object

In [95]:
df_PHEV['Unnamed: 44'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                      NaN
8                                     1000
9                                     1000
Name: Unnamed: 44, dtype: object

In [96]:
df_FCV['Unnamed: 44'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                      NaN
8                                     1250
9                                     1250
Name: Unnamed: 44, dtype: object

In [97]:
df_EV = df_EV.drop(df_EV.index[[4,5]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[4,5]])
df_FCV = df_FCV.drop(df_FCV.index[[4,5]])
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 44'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()
OutPut = OutPut.append({'Year':2018, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

In [98]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.905660,698.125000,1318.604651,1300.0
1,2021.0,2095.423729,695.098039,1492.000000,1320.0
2,2020.0,2076.817793,646.052632,1445.555556,1350.0
3,2019.0,1904.559748,628.571429,1258.333333,1350.0
4,2018.0,1778.682171,618.750000,1213.235294,1250.0


# Dataset in 2017

In [99]:
df_gas = pd.read_excel('../Raw_Data/2017 FE Guide for DOE-release dates before 3-25-2019-no sales-9-19-2019McLarenforpublic.xlsx', sheet_name = 'FEguide')
df_EV = pd.read_excel('../Raw_Data/2017 FE Guide for DOE-release dates before 3-25-2019-no sales-9-19-2019McLarenforpublic.xlsx', sheet_name = 'EVs')
df_PHEV = pd.read_excel('../Raw_Data/2017 FE Guide for DOE-release dates before 3-25-2019-no sales-9-19-2019McLarenforpublic.xlsx', sheet_name = 'PHEVs')
df_FCV = pd.read_excel('../Raw_Data/2017 FE Guide for DOE-release dates before 3-25-2019-no sales-9-19-2019McLarenforpublic.xlsx', sheet_name = 'FCVs')

In [100]:
df_FCV['Unnamed: 45'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                                      NaN
5                            Ann Fuel Cost
6    Annual Fuel1 Cost - Conventional Fuel
7                                      NaN
8                                     1250
9                                     1250
Name: Unnamed: 45, dtype: object

In [101]:
df_EV = df_EV.drop(df_EV.index[[4,5]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[4,5]])
df_FCV = df_FCV.drop(df_FCV.index[[5,6]])
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 44'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 45'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 45'].dropna().mean()
OutPut = OutPut.append({'Year':2017, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

In [102]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.905660,698.125000,1318.604651,1300.0
1,2021.0,2095.423729,695.098039,1492.000000,1320.0
2,2020.0,2076.817793,646.052632,1445.555556,1350.0
3,2019.0,1904.559748,628.571429,1258.333333,1350.0
4,2018.0,1778.682171,618.750000,1213.235294,1250.0
5,2017.0,1794.774920,645.000000,1131.578947,1400.0


# Dataset in 2016

In [103]:
df_gas = pd.read_excel('../Raw_Data/2016 FEGuide for DOE-OK to release-no-sales-5-8-2019_Mercedes_public.xlsx', sheet_name = 'FEguide')
df_EV = pd.read_excel('../Raw_Data/2016 FEGuide for DOE-OK to release-no-sales-5-8-2019_Mercedes_public.xlsx', sheet_name = 'EVs')
df_PHEV = pd.read_excel('../Raw_Data/2016 FEGuide for DOE-OK to release-no-sales-5-8-2019_Mercedes_public.xlsx', sheet_name = 'PHEVs')
df_FCV = pd.read_excel('../Raw_Data/2016 FEGuide for DOE-OK to release-no-sales-5-8-2019_Mercedes_public.xlsx', sheet_name = 'FCVs')

In [104]:
df_FCV['Unnamed: 45'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                                      NaN
5                            Ann Fuel Cost
6    Annual Fuel1 Cost - Conventional Fuel
7                                      NaN
8                                     1700
9                                     1700
Name: Unnamed: 45, dtype: object

In [105]:
df_EV = df_EV.drop(df_EV.index[[5,6]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[6,7]])
df_FCV = df_FCV.drop(df_FCV.index[[5,6]])
cost_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
cost_of_EV = df_EV['Unnamed: 45'].dropna().mean()
cost_of_PHEV = df_PHEV['Unnamed: 45'].dropna().mean()
cost_of_FCV = df_FCV['Unnamed: 45'].dropna().mean()
OutPut = OutPut.append({'Year':2016, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

In [106]:
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV
0,2022.0,1859.905660,698.125000,1318.604651,1300.0
1,2021.0,2095.423729,695.098039,1492.000000,1320.0
2,2020.0,2076.817793,646.052632,1445.555556,1350.0
3,2019.0,1904.559748,628.571429,1258.333333,1350.0
4,2018.0,1778.682171,618.750000,1213.235294,1250.0
5,2017.0,1794.774920,645.000000,1131.578947,1400.0
6,2016.0,2147.732894,664.516129,1311.111111,1475.0


# Dataset in 2015

In [107]:
df_gas = pd.read_excel('../Raw_Data/2015 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'FEguide')
df_EV = pd.read_excel('../Raw_Data/2015 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'EVs')
df_PHEV = pd.read_excel('../Raw_Data/2015 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'PHEVs')
df_FCV = pd.read_excel('../Raw_Data/2015 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'Fuel cell vehs')

In [108]:
df_FCV['Unnamed: 44'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                     1700
8                                     1700
Name: Unnamed: 44, dtype: object

In [109]:
df_EV = df_EV.drop(df_EV.index[[3,4]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[3,4]])
df_FCV = df_FCV.drop(df_FCV.index[[4,5]])
Save_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
Save_of_EV = df_EV['Unnamed: 45'].dropna().mean()
Save_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
Save_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()
OutPut = OutPut.append({'Year':2015, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

# Dataset in 2014

In [110]:
df_gas = pd.read_excel('../Raw_Data/2014 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'FE Guide')
df_EV = pd.read_excel('../Raw_Data/2014 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'EVs')
df_PHEV = pd.read_excel('../Raw_Data/2014 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'PHEVs')
df_FCV = pd.read_excel('../Raw_Data/2014 FEGuide for DOE-OK to release-no-sales-5-8-2019_updated_Mercedes_public.xlsx', sheet_name = 'FCVs')

In [111]:
df_FCV['Unnamed: 44'].head(10)

0                                      NaN
1                                      NaN
2                                      NaN
3                                      NaN
4                            Ann Fuel Cost
5    Annual Fuel1 Cost - Conventional Fuel
6                                      NaN
7                                     1400
8                                     1400
9                                      NaN
Name: Unnamed: 44, dtype: object

In [112]:
df_EV = df_EV.drop(df_EV.index[[5,6]])
df_PHEV = df_PHEV.drop(df_PHEV.index[[3,4]])
df_FCV = df_FCV.drop(df_FCV.index[[4,5]])
Save_of_gas = df_gas['Annual Fuel1 Cost - Conventional Fuel'].dropna().mean()
Save_of_EV = df_EV['Unnamed: 44'].dropna().mean()
Save_of_PHEV = df_PHEV['Unnamed: 44'].dropna().mean()
Save_of_FCV = df_FCV['Unnamed: 44'].dropna().mean()
OutPut = OutPut.append({'Year':2014, 'cost_of_gas':cost_of_gas, 'cost_of_EV':cost_of_EV, 'cost_of_PHEV':cost_of_PHEV, 'cost_of_FCV':cost_of_FCV}, ignore_index=True)

# OUTPUT

In [113]:
OutPut['Avg_NE'] = OutPut.iloc[:, 2:5].mean(axis=1)
OutPut

,Year,cost_of_gas,cost_of_EV,cost_of_PHEV,cost_of_FCV,Avg_NE
0,2022.0,1859.905660,698.125000,1318.604651,1300.0,1105.576550
1,2021.0,2095.423729,695.098039,1492.000000,1320.0,1169.032680
2,2020.0,2076.817793,646.052632,1445.555556,1350.0,1147.202729
3,2019.0,1904.559748,628.571429,1258.333333,1350.0,1078.968254
4,2018.0,1778.682171,618.750000,1213.235294,1250.0,1027.328431
5,2017.0,1794.774920,645.000000,1131.578947,1400.0,1058.859649
6,2016.0,2147.732894,664.516129,1311.111111,1475.0,1150.209080
7,2015.0,2147.732894,664.516129,1311.111111,1475.0,1150.209080
8,2014.0,2147.732894,664.516129,1311.111111,1475.0,1150.209080


In [114]:
path = '../Processed_Data/Save_money.csv'
OutPut.to_csv(path)